In [ ]:
import os
import re
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="chromadb")

c:\Users\solon\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Load Data

In [2]:
df = pd.read_csv(r"G:\Other computers\My Laptop\Drive\Magang\DataIns\RAG\data\processed\extracted_data_sahabatai.csv")
# Tambahkan kolom id
df.insert(0, 'id', range(1, 1 + len(df)))

In [3]:
df = df[['id', 'Kota', 'Akun Instagram', 'Kategori Tempat', 'deskripsi', 'opini']]

In [4]:
df

,id,Kota,Akun Instagram,Kategori Tempat,deskripsi,opini
0,1,Sleman,@lyonscafe.co,Ngopi Santai,"Coffee shop luas dengan area parkir yang luas,...",Positif (menguji tempat duduk dan menu yang te...
1,2,Sleman,@berikopi.jogja,Ngopi Santai,Coffee shop baru di Bandung dengan menu yang e...,Tempat dan menu enak.
2,3,Yogyakarta,"@elder.onthetable, @elderpatisserie",Ngopi Santai,.onthetable adalah coffee shop yang baru buka ...,Tempat ini memiliki suasana yang nyaman dan me...
3,4,Yogyakarta,@coldnbrew,Area Lengkap,"Cold N Brew baru buka cabang di Malioboro, buk...",Netral.
4,5,Yogyakarta,@waktuluang.deloji,Ngopi Santai,"Coffee shop bernama .deloji di Jogja, dekat ti...","Positif, penulis memuji konsep, menu, dan pela..."
...,...,...,...,...,...,...
304,305,Lainnya,@lunariacoffee,WFC Nyaman,"Lunaria, coffee and eatery estetik baru di Ged...","Suasana kondusif dan tenang, didukung oleh ker..."
305,306,Sleman,@kolokial.coffeeyk,Menu Variatif,Tempat nongkrong baru di pinggir jalan.,Netral.
306,307,Sleman,@kopiruangrindu,WFC Nyaman,Coffee shop dengan konsep coffee bar bertema g...,"Suasana coffee shop ini santai dan nostalgic, ..."
307,308,Yogyakarta,@kastemspace_gantara,Menu Variatif,"Kastem Space Gantara, coffee shop di selatan J...",Positif. Penulis menyukai suasana dan menu di ...


In [5]:
# Ganti nama kolom dengan replacement yang diinginkan
df.columns = ['id', 'lokasi', 'source', 'kategori', 'deskripsi', 'opini']

## 2. Cleaning Function

In [6]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r"\s+", " ", text)  # hapus spasi berlebih
    return text.strip()

for col in ["kategori", "lokasi", "source", "deskripsi", "opini"]:
    df[col] = df[col].apply(clean_text)

## 3. Build Content Field

In [7]:
def build_content(row):
    return f"""
Kategori: {row['kategori']}
Lokasi: {row['lokasi']}
Sumber: {row['source']}

Deskripsi:
{row['deskripsi']}

Opini:
{row['opini']}
""".strip()

df["content"] = df.apply(build_content, axis=1)

## 4. Convert to LangChain Documents

In [8]:
raw_documents = []
for _, row in df.iterrows():
    raw_documents.append(
        Document(
            page_content=row["content"],
            metadata={
                "id": row["id"],
                "kategori": row["kategori"],
                "lokasi": row["lokasi"],
                "source": row["source"],
            },
        )
    )

## 5. Chunking

In [9]:
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=400,
#     chunk_overlap=80,
#     separators=["\n\n", "\n", ".", " "]
# )
# 
# documents = text_splitter.split_documents(raw_documents)
# 
# print(f"Total chunks created: {len(documents)}")

## 6. Load Embedding Model

In [10]:
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

class Embedding:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return self.model.encode(
            texts,
            batch_size=32,
            show_progress_bar=True,
            normalize_embeddings=True
        ).tolist()

    def embed_query(self, text):
        return self.model.encode(
            f"query: {text}",
            normalize_embeddings=True
        ).tolist()

embedding_function = Embedding(embedding_model)

Loading weights: 100%|██████████| 391/391 [00:00<00:00, 1444.80it/s, Materializing param=pooler.dense.weight]                               
XLMRobertaModel LOAD REPORT from: intfloat/multilingual-e5-large
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


## 7. Store to Chroma DB

In [11]:
persist_dir = r"G:\Other computers\My Laptop\Drive\Magang\DataIns\RAG\data\vector_store\chroma_db"

# Hapus koleksi lama agar dimensi embedding tidak bentrok
if os.path.exists(persist_dir):
    for root, dirs, files in os.walk(persist_dir, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir(persist_dir)

vectorstore = Chroma.from_documents(
    documents=raw_documents,
    embedding=embedding_function,
    persist_directory=persist_dir
)

vectorstore.persist()

print("Vector DB tersimpan")

Batches: 100%|██████████| 10/10 [00:02<00:00,  4.29it/s]


Vector DB tersimpan


C:\Users\solon\AppData\Local\Temp\ipykernel_42128\3813646481.py:18: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## 8. Test Retrieval

In [12]:
query = "Rekomendasikan tempat kopi yang nongkrong dan makan"
results = vectorstore.similarity_search(query, k=5)

print("\nHasil Retrieval\n")
for i, r in enumerate(results, 1):
    print(f"Result {i}")
    print(r.page_content)
    print("Metadata:", r.metadata)
    print("-" * 50)


Hasil Retrieval

Result 1
Kategori: Ngopi Santai
Lokasi: Sleman
Sumber: @nara_kupu_jogja

Deskripsi:
Coffee shop ini gratis untuk masuk, bisa makan, nongkrong, atau memberi makan rusa. Tiket untuk memberi makan rusa dari luar pagar kandang dan masuk ke dalam kandang. Tempat luas bisa untuk outbond, gathering, atau acara lainnya.

Opini:
Netral
Metadata: {'source': '@nara_kupu_jogja', 'id': 139, 'lokasi': 'Sleman', 'kategori': 'Ngopi Santai'}
--------------------------------------------------
Result 2
Kategori: Ngopi Santai
Lokasi: Lainnya
Sumber: @daongsignature.ketep

Deskripsi:
Coffee shop dengan konsep joglo, indoor dan outdoor, pemandangan kaki gunung dan udara dingin khas Ketep, buka setiap hari.

Opini:
Positif (menikmati pemandangan, suasana, dan menu yang beragam).
Metadata: {'lokasi': 'Lainnya', 'source': '@daongsignature.ketep', 'kategori': 'Ngopi Santai', 'id': 236}
--------------------------------------------------
Result 3
Kategori: Ngopi Santai
Lokasi: Sleman
Sumber: @wa

In [13]:
# print result ketiga
print(f"Result 3")
print(results[2].page_content)

Result 3
Kategori: Ngopi Santai
Lokasi: Sleman
Sumber: @warungkulinan, @warungkulinan.

Deskripsi:
Coffee shop dengan konsep rimbun dan banyak pepohonan hijau, cocok untuk bekerja, minum kopi, dan makan. Tersedia berbagai pilihan menu mulai dari kopi, snack tradisional, penyetan, bakmi jawa, seafood, hingga bubur kacang ijo dengan harga terjangkau.

Opini:
Suasana tenang dan nyaman dengan suara gemercik air di bagian belakang.


## 9. (Optional) Load DB Again Without Re-embedding

In [14]:
# vectorstore = Chroma(
#     persist_directory=persist_dir,
#     embedding_function=embedding_function
# )

# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})